공식 깃허브: https://github.com/LouisChen1992/Deep_Learning_Asset_Pricing

# Colab Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/MyDrive/000research/return_prediction/최종/01_전처리'
os.chdir(path)

In [ ]:
pip install tensorflow==1.14.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 49 kB/s 
     |████████████████████████████████| 50 kB 5.8 MB/s 
     |████████████████████████████████| 488 kB 43.7 MB/s 
     |████████████████████████████████| 3.1 MB 37.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
    Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949
ERROR: pip's dependency resolver does not currently ta

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)

from src.data import data_layer
from src.utils import deco_print

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# LoadData

In [ ]:
mac = np.load('./raw_data/macroeconomic/mac2022.npz', allow_pickle=True)
mac['data'].shape

(316, 127)

In [ ]:
class LoadData:
    def __init__(self, pathMacroFeature, start_year, end_year):
        self._UNK = -99.99
        self._start_year = start_year
        self._end_year = end_year
        self._load_macro_feature(pathMacroFeature)
        
    def _load_macro_feature(self, pathMacroFeature):
        tmp = np.load(pathMacroFeature, allow_pickle=True)
        dates = tmp['date']

        self._years = np.array([int(str(s)[:4]) for s in dates])
        self._year_mask = ( (self._years >= self._start_year) & (self._years <= self._end_year) )

        macro_idx = np.arange(len(tmp['variable']))
        self._macroFeature = tmp['data'][:,macro_idx]
        self._year_masked_macroFeature = self._macroFeature[self._year_mask]

    def iterateOneEpoch(self, subEpoch=False):
        if subEpoch:
            for _ in range(subEpoch):
                yield self._year_masked_macroFeature
        else:
            yield self._year_masked_macroFeature

In [ ]:
dl = LoadData(
            pathMacroFeature='./raw_data/macroeconomic/mac2022.npz',
            start_year=1996, 
            end_year=2022)

for _, (I_macro) in enumerate(dl.iterateOneEpoch(subEpoch=False)):
    print('dl')
    print(I_macro.shape)

dl
(316, 127)


# Config

In [ ]:
#### Delete all flags before declare #####

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

- tensorflow의 flag란?
- https://daeson.tistory.com/256

In [ ]:
del_all_flags(tf.flags.FLAGS)
tf.flags.DEFINE_string("f", "", "kernel") # 주피터 노트북에서 돌릴땐 이 코드 추가해야 돌아감

tf.flags.DEFINE_string('config', path+'/config/config.json', 'Path to the file with configurations')
tf.flags.DEFINE_string('logdir', '', 'Path to save logs and checkpoints')

tf.flags.DEFINE_integer('saveBestFreq', -1, 'Frequency to save best model')
tf.flags.DEFINE_boolean('printOnConsole', True, 'Print on console or not')
tf.flags.DEFINE_boolean('saveLog', True, 'Save log or not')
tf.flags.DEFINE_integer('printFreq', 128, 'Frequency to print on console')
tf.flags.DEFINE_integer('ignoreEpoch', 64, 'Ignore first several epochs')

FLAGS = tf.flags.FLAGS

In [ ]:
with open(FLAGS.config, 'r') as file:
    config = json.load(file)
    if not 'macro_idx' in config:
        config['macro_idx'] = None

deco_print('Read the following in config: ')
print(json.dumps(config, indent=4))

>==================> Read the following in config: 
{
    "num_epochs": 100,
    "learning_rate": 0.01,
    "optimizer": "Adam",
    "num_layers": 2,
    "hidden_dim": [
        64,
        64,
        64,
        64
    ],
    "num_layers_moment": 0,
    "num_units_rnn": [
        4
    ],
    "macro_feature_file_test": "datasets/macro/macro_test.npz",
    "macro_feature_file": "datasets/macro/macro_train.npz",
    "use_rnn": true,
    "weighted_loss": true,
    "cell_type_rnn_moment": "lstm",
    "cell_type_rnn": "lstm",
    "macro_feature_file_valid": "datasets/macro/macro_valid.npz",
    "num_epochs_moment": 64,
    "tSize": 192,
    "individual_feature_dim": 48,
    "macro_feature_dim": 127,
    "num_condition_moment": 8,
    "loss_factor": 1.0,
    "num_layers_rnn_moment": 1,
    "individual_feature_file_valid": "datasets/char/Char_valid.npz",
    "individual_feature_file": "datasets/char/Char_train.npz",
    "num_units_rnn_moment": [
        32
    ],
    "num_epochs_unc": 256,


# get hidden states

In [ ]:
from tensorflow.python.ops.rnn_cell import BasicRNNCell
from tensorflow.python.ops.rnn_cell import GRUCell
from tensorflow.python.ops.rnn_cell import LSTMCell
from tensorflow.python.ops.rnn_cell import DropoutWrapper
from tensorflow.python.ops.rnn_cell import MultiRNNCell

def create_rnn_cell(cell_type, num_units, num_layers=1, dp_input_keep_prob=1.0, dp_output_keep_prob=1.0):
	def single_cell(num_units):
		if cell_type == 'rnn':
			cell_class = BasicRNNCell
		elif cell_type == 'gru':
			cell_class = GRUCell
		elif cell_type == 'lstm':
			cell_class = LSTMCell
		else:
			raise ValueError('Cell Type Not Supported! ')

		if dp_input_keep_prob != 1.0 or dp_output_keep_prob != 1.0:
			return DropoutWrapper(cell_class(num_units=num_units),
				input_keep_prob=dp_input_keep_prob,
				output_keep_prob=dp_output_keep_prob)
		else:
			return cell_class(num_units=num_units)

	assert(len(num_units) == num_layers)
	if num_layers > 1:
		return MultiRNNCell([single_cell(num_units[i]) for i in range(num_layers)])
	else:
		return single_cell(num_units[0])

In [ ]:
def initial_state_size(cell_type, num_units):
	state_size = sum(num_units)
	return state_size * 2

In [ ]:
from tensorflow.python.ops.rnn_cell_impl import LSTMStateTuple

class HiddenStates():
    '''
    macSize : 316
    '''
    def __init__(self, model_params, macSize=316, force_var_reuse=False, global_step=None):
        # super(Model, self).__init__(model_params, global_step)
        self._force_var_reuse = force_var_reuse
        self.model_params = model_params
        self._global_step = global_step

        # ---- placeholders ---- #
        self._macSize = macSize
        self._macro_feature_dim = self.model_params['macro_feature_dim']
        self._I_macro_placeholder = tf.placeholder(dtype=tf.float32, shape=[self._macSize, self._macro_feature_dim], name='macroFeaturePlaceholder')
        self._dropout_placeholder = tf.placeholder_with_default(1.0, shape=[], name='Dropout')
        
        ### rnn input ###
        _I_macro_placeholder = self._I_macro_placeholder ; print('_I_macro_placeholder', _I_macro_placeholder.shape)
        self._rnn_input = tf.expand_dims(self._I_macro_placeholder, axis=0) 
        rnn_input = self._rnn_input ; print('rnn_input: ', rnn_input.shape)

        with tf.variable_scope(name_or_scope='Model_Layer', reuse=self._force_var_reuse):
            self.build()

    def build(self):

        with tf.variable_scope('RNN_Layer'):
            rnn_cell = create_rnn_cell(
                        cell_type=self.model_params['cell_type_rnn'], # lstm
                        num_units=self.model_params['num_units_rnn'], # 4
                        num_layers=self.model_params['num_layers_rnn'], # 1
                        dp_input_keep_prob=self._dropout_placeholder,
                        dp_output_keep_prob=1.0
                        )

            rnn_outputs, rnn_state = tf.nn.dynamic_rnn(
                        cell=rnn_cell,
                        inputs=self._rnn_input,
                        # initial_state=self._initial_state,
                        dtype=tf.float32)
            self._rnn_state = rnn_state
            self._rnn_outputs = rnn_outputs
            print('rnn_outputs: ', rnn_outputs.shape)
            
            self._macro_nn_input = tf.squeeze(rnn_outputs, axis=0) # RNN 통과해서 나온 macro 변수들의 hidden state
            _macro_nn_input = self._macro_nn_input ; print('_macro_nn_input', _macro_nn_input.shape)

    def getHiddenStates(self, sess, dl):
        ###### initialize variables ######
        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        sess.run(init_g)
        sess.run(init_l)

        for _, (I_macro) in enumerate(dl.iterateOneEpoch(subEpoch=False)):
            feed_dict = {self._I_macro_placeholder:I_macro,
                         self._dropout_placeholder:1.0
                        }
            hidden_states = sess.run(fetches=[self._macro_nn_input], feed_dict=feed_dict)
            return hidden_states

In [ ]:
model_params = config
tf.reset_default_graph()
global_step = tf.train.get_or_create_global_step()
hs_model = HiddenStates(model_params=config, macSize=316, global_step=global_step)

_I_macro_placeholder (316, 127)
rnn_input:  (1, 316, 127)
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
rnn_outputs:  (1, 316, 4)
_macro_nn_input (316, 4)


In [ ]:
sess = tf.Session()
hidden_states = hs_model.getHiddenStates(sess, dl)
print(hidden_states[0].shape)

(316, 4)


In [ ]:
import pandas as pd
hidden_states_df = pd.DataFrame(hidden_states[0])

hidden_states_df.rename({0:'hs1', 1:'hs2', 2:'hs3', 3:'hs4'}, axis=1, inplace=True)

In [ ]:
lis = pd.date_range(start="1996-01-01", end="2022-04-30", freq='M') 
lis = [date_obj.strftime('%Y-%m-%d') for date_obj in lis]
lis = pd.DataFrame({'date':lis})
lis['date'] = pd.to_datetime(lis['date'])

In [ ]:
lis

,date
0,1996-01-31
1,1996-02-29
2,1996-03-31
3,1996-04-30
4,1996-05-31
...,...
311,2021-12-31
312,2022-01-31
313,2022-02-28
314,2022-03-31


In [ ]:
hidden_states_df = pd.concat([lis, hidden_states_df], axis=1)

In [ ]:
hidden_states_df

,date,hs1,hs2,hs3,hs4
0,1996-01-31,0.022903,-0.143527,-0.075930,0.115051
1,1996-02-29,-0.084758,-0.191410,-0.100266,0.177098
2,1996-03-31,-0.074154,-0.174632,-0.104189,0.196434
3,1996-04-30,0.017234,-0.075972,-0.138881,0.225288
4,1996-05-31,0.044322,-0.456727,-0.163345,0.094094
...,...,...,...,...,...
311,2021-12-31,-0.039253,-0.119768,-0.137271,0.083944
312,2022-01-31,-0.081208,-0.207680,-0.202407,0.118454
313,2022-02-28,-0.064052,-0.107213,-0.121080,0.141095
314,2022-03-31,-0.021913,-0.104888,-0.178740,0.176583


In [ ]:
hidden_states_df.to_csv('hidden_states_df2022.csv', index=False)